# **Selenium Python Framework Implementation**

- Standard of writing selenium tests in framework
- Creating browser invocation Fixtures in confest.py
- Setting up base class to hold all common Utilities
- Inheriting Base class to all tests to remove fixture redundant Code
- Passing command line options to select browser at run time
- Implementing page object mechanism
- Smarter way of returning page objects from navigation methods
- Creating selenium webdriver custom utilities in base class
- Parameterising webdriver tests with multiple data sets
- Organizing data from separate data files and injecting into fixture at run time
- Implementing logging feature to webdriver tests 
- Text execution HTML reporting
- Automatic screenshot capture on test failures
- Integrating selenium python framework to jenkins CI tool with jenkin build Parameterization
- Github basics for project version control


To create a robust **Selenium Python Test Automation Framework** using **Pytest**, **Jenkins**, and **GitHub**, follow this step-by-step guide that addresses all the points you mentioned. This will help you build a maintainable, scalable, and efficient automation framework.

## **1. Setting Up the Project Structure**

Start by setting up a proper folder structure for your Selenium-Pytest framework:

```
selenium_pytest_framework/
│
├── tests/                   # Holds all test files
│   ├── test_sample.py
│
├── pages/                   # Holds page object model files
│   ├── base_page.py
│   ├── login_page.py
│
├── utils/                   # Custom utilities (common methods, logging)
│   ├── logger.py
│   ├── data_utils.py
│
├── data/                    # Data files for parameterized tests
│   ├── test_data.json
│
├── reports/                 # Test reports and screenshots
│
├── screenshots/             # Stores screenshots on failures
│
├── conftest.py              # Fixtures and setup code
│
├── requirements.txt         # All project dependencies
│
├── pytest.ini               # Pytest configuration
│
├── Jenkinsfile              # Jenkins build file
│
└── README.md                # Project documentation
```

## **2. Standard of Writing Selenium Tests in Framework**

In each test file, follow a structured format where:
- You follow the **Arrange-Act-Assert** pattern.
- Use **Page Object Model (POM)** for managing locators and actions for each page.

```python
from pages.login_page import LoginPage

def test_login_valid_user(init_browser):
    login_page = LoginPage(init_browser)
    
    # Arrange
    login_page.navigate_to_login()
    
    # Act
    login_page.login('user', 'password')

    # Assert
    assert login_page.is_login_successful()
```

## **3. Browser Invocation Fixtures in `conftest.py`**

You need to use **pytest fixtures** in `conftest.py` to initialize the WebDriver and clean up after test execution.

```python
import pytest
from selenium import webdriver

@pytest.fixture(params=["chrome", "firefox"], scope='class')
def init_browser(request):
    browser = request.param
    if browser == "chrome":
        driver = webdriver.Chrome()
    elif browser == "firefox":
        driver = webdriver.Firefox()
    driver.maximize_window()
    driver.implicitly_wait(10)
    
    request.cls.driver = driver
    yield driver
    driver.quit()
```
In the provided `pytest` fixture, `request` is an object of the `pytest.FixtureRequest` class, which is passed as an argument to the fixture function. It provides access to information about the test function or class that is currently being executed. Here's how `request` is used in your code:

### Key Uses of `request` in the Fixture:

1. **Accessing the `param` attribute**:
   - `request.param` allows access to the parameters that are passed to the fixture. In this case, the fixture is parameterized with `params=["chrome", "firefox"]`, which means `request.param` will contain either `"chrome"` or `"firefox"`, depending on which test is being run.
   
   Example:
   ```python
   browser = request.param  # 'chrome' or 'firefox' based on the test run
   ```

2. **Setting the `driver` attribute on the class (`request.cls`)**:
   - `request.cls.driver = driver` assigns the `driver` object (the WebDriver instance) to the test class. This allows the WebDriver to be accessible to all test methods within the class where this fixture is used.

   Example:
   ```python
   request.cls.driver = driver  # Makes the driver accessible to the test class
   ```

### Breakdown of the Code:
- `@pytest.fixture`: Marks the function as a fixture, and it can be shared across multiple test functions or classes.
- `params=["chrome", "firefox"]`: The fixture will run twice for each test that uses it, once with `"chrome"` and once with `"firefox"`.
- `request.param`: Refers to the browser type, which is either `"chrome"` or `"firefox"`.
- `request.cls`: Refers to the test class that is using this fixture. By assigning `driver` to `request.cls.driver`, the WebDriver is made available in the test class as `self.driver`.

So, `request` helps in:
- Providing the parameter values passed to the fixture.
- Interacting with the test class to inject or share resources like the WebDriver across test methods.

## **4. Base Class for Common Utilities**

The **base class** holds common methods that can be reused across different test cases, such as navigation, waits, screenshot capture, etc.

```python
class BasePage:
    def __init__(self, driver):
        self.driver = driver

    def capture_screenshot(self, name):
        self.driver.save_screenshot(f"./screenshots/{name}.png")

    def wait_for_element(self, by, value, timeout=10):
        WebDriverWait(self.driver, timeout).until(EC.presence_of_element_located((by, value)))
```

## **5. Inherit Base Class in All Tests**

In each test class, inherit the `BasePage` to avoid redundant code in your test files:

```python
class LoginPage(BasePage):
    def __init__(self, driver):
        super().__init__(driver)

    def navigate_to_login(self):
        self.driver.get("https://example.com/login")

    def login(self, username, password):
        self.driver.find_element(By.ID, "user").send_keys(username)
        self.driver.find_element(By.ID, "pass").send_keys(password)
        self.driver.find_element(By.ID, "login").click()

    def is_login_successful(self):
        return "Dashboard" in self.driver.title
```

## **6. Command Line Options for Browser Selection**

Allow selecting the browser type dynamically via command-line options:

```python
def pytest_addoption(parser):
    parser.addoption("--browser", action="store", default="chrome", help="Browser to run tests")

@pytest.fixture(scope="class")
def init_browser(request):
    browser = request.config.getoption("--browser")
    if browser == "chrome":
        driver = webdriver.Chrome()
    elif browser == "firefox":
        driver = webdriver.Firefox()
    driver.maximize_window()
    driver.implicitly_wait(10)
    request.cls.driver = driver
    yield driver
    driver.quit()
```

To run tests with a specific browser, use:
```bash
pytest --browser firefox
```

## **7. Page Object Mechanism**

Follow the **Page Object Model** (POM) for separating UI actions and elements into dedicated classes for each page of your application. This improves maintainability.

## **8. Returning Page Objects from Navigation Methods**

Use method chaining to navigate through pages and return page objects for smooth transitions:

```python
def login(self, username, password):
    self.driver.find_element(By.ID, "user").send_keys(username)
    self.driver.find_element(By.ID, "pass").send_keys(password)
    self.driver.find_element(By.ID, "login").click()
    return DashboardPage(self.driver)
```

## **9. Custom Selenium WebDriver Utilities**

Add custom WebDriver utilities such as scrolling, taking screenshots, or waiting for elements in the **BasePage** class for reuse.

## **10. Parameterizing WebDriver Tests with Multiple Data Sets**

Use **pytest.mark.parametrize** to provide multiple data sets to a single test:

```python
@pytest.mark.parametrize("username, password", [
    ("user1", "password1"),
    ("user2", "password2")
])
def test_login(init_browser, username, password):
    login_page = LoginPage(init_browser)
    login_page.login(username, password)
    assert login_page.is_login_successful()
```

## **11. Organize Data from Separate Files**

Store test data in separate files like JSON or CSV and inject the data into the test fixture dynamically:

```python
import json

@pytest.fixture(params=json.load(open('data/test_data.json')))
def get_data(request):
    return request.param
```

## **12. Implement Logging Feature**

Create a logging utility using Python’s `logging` module and integrate it into your framework to track test execution.

```python
import logging

def get_logger():
    logger = logging.getLogger(__name__)
    file_handler = logging.FileHandler('test.log')
    formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.setLevel(logging.INFO)
    return logger
```

## **13. HTML Reporting with Pytest**

Use **pytest-html** for generating HTML reports after test execution.

Install the plugin:
```bash
pip install pytest-html
```

Run tests with HTML report generation:
```bash
pytest --html=reports/test_report.html
```

## **14. Automatic Screenshot Capture on Test Failures**

Add a hook in `conftest.py` to capture screenshots automatically when a test fails:

```python
@pytest.hookimpl(tryfirst=True, hookwrapper=True)
def pytest_runtest_makereport(item, call):
    outcome = yield
    report = outcome.get_result()
    if report.when == 'call' and report.failed:
        driver = item.instance.driver
        driver.save_screenshot(f"screenshots/{item.name}.png")
```

## **15. Jenkins Integration with Build Parameterization**

1. **Install Jenkins** and add the **Pytest plugin**.
2. Create a **Jenkinsfile** for pipeline execution:
   ```groovy
   pipeline {
       agent any
       stages {
           stage('Test') {
               steps {
                   sh 'pytest --html=reports/test_report.html'
               }
           }
       }
   }
   ```
3. Use Jenkins **build parameters** to run tests on different browsers or environments.

## **16. GitHub for Version Control**

1. Initialize a Git repository:
   ```bash
   git init
   git add .
   git commit -m "Initial commit"
   ```
2. Push the code to GitHub:
   ```bash
   git remote add origin <remote_url>
   git push -u origin master
   ```

This guide provides a robust framework with **Selenium, Pytest, Page Object Model, Jenkins**, and **GitHub** integration. It also includes parameterization, logging, HTML reporting, and automatic screenshot capture on test failures.

---